# 在Notebook中使用Plotly绘图

Plotly是一套Javascript绘图库，它能够绘制曲线图、散列图、柱状图、三维曲线、曲面等多种图表。它提供了Python调用接口，可以使用Python生成绘图所需的Javascript代码。在Python中有在线和离线两种方式。在线方式需要用户申请Plotly的帐号，所有生成的图表都会保存在Plotly的服务器中。而离线方式则是在页面中插入Plotly提供的Javascript库，然后调用该库提供的API函数绘图。在命令行中可以输入下面的命令安装`plotly`库。

```
pip install plotly
```

然后在Notebook中执行如下代码初始化离线的Notebook模式。

```python
from plotly.offline import init_notebook_mode
init_notebook_mode()
```

调用`init_notebook_mode()`会将Plotly的整个Javascript库插入到Notebook的单元输出之中，这样做会增加Notebook的字节数，因此本节提供两种引用Plotly的Javascript库的方法，能极大地减小Notebook文件的大小。

下面的`init_plotly_online_mode()`从Plotly的网站上载入Plotly库。

In [ ]:
def init_plotly_online_mode():
    from IPython.display import display_javascript
    from plotly import offline
    offline.offline.__PLOTLY_OFFLINE_INITIALIZED = True
    jscode = """
    require.config({
      paths: {
        d3: 'http://cdnjs.cloudflare.com/ajax/libs/d3/3.5.16/d3.min',
        plotly: 'http://cdn.plot.ly/plotly-1.10.0.min'
      },

      shim: {
        plotly: {
          deps: ['d3', 'jquery'],
          exports: 'plotly'
        }
      }
    });

    require(['d3', 'plotly'], function(d3, plotly) {
        window.Plotly = plotly;
    });
    """
    display_javascript(jscode, raw=True)

下面的`init_plotly_offline_mode()`将`plotly`库自带的`plotly.min.js`复制到与Notebook相同路径之下，并在客户端载入该文件：

In [44]:
def init_plotly_offline_mode():
    from IPython.display import display_javascript
    import shutil
    from plotly import offline
    from os import path
    offline.offline.__PLOTLY_OFFLINE_INITIALIZED = True
    shutil.copy(path.join(offline.__path__[0], "plotly.min.js"), ".")

    jscode = """
    require.config({
      paths: {
        plotly: "/files/" + Jupyter.notebook.notebook_path + "/../" + "plotly.min"
      },

      shim: {
        plotly: {
          deps: [],
          exports: 'plotly'
        }
      }
    });

    require(['plotly'], function(plotly) {
        window.Plotly = plotly;
        console.log("Plotly loaded");
    });
    """
    display_javascript(jscode, raw=True)

如果一切正常，运行下面的代码会在客户端载入Plotly库，并在调试窗口中显示`Plotly loaded`。

In [45]:
init_plotly_offline_mode()

下面调用`iplot()`创建一幅曲线图，它的唯一参数是一个描述图表的字典，其中`data`键对应一个包含两条曲线的列表，`layout`键对应描述图表属性的字典。每条曲线也是由字典描述的，其`x`和`y`键对应曲线上各点的坐标。

In [51]:
from plotly.offline import iplot
import numpy as np

x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)
y2 = y + np.random.normal(0, 0.1, len(x))

line = {"x":x, "y":y, "name":"sin", "line":{"width":3, "color":"blue"}, "opacity":0.5}
line2 = {"x":x, "y":y2, "name":"sin + noise", "opacity":0.7}
layout = {"title": "Plotly Example", "width":600, "height":400}
fig = {"data":[line, line2], "layout":layout}
iplot(fig)

## 使用`cufflinks`库快速绘图

使用`iplot()`绘图需要用户自己创建表示图表的字典，所需的代码量比较大。如果读者希望快速绘图，可以使用`cufflinks`。
该绘图库为`pandas`的`DataFrame`对象提供了`iplot()`方法。`

调用下面的命令安装`cufflinks`:

```pip install cufflinks```

为了让`cufflinks`使用离线方式绘图，需要调用其`go_offline()`函数：

In [53]:
import cufflinks
cufflinks.go_offline()

接下来创建`DataFrame`对象，并调用其`iplot()`方法绘图。注意`iplot()`是`cufflinks`为`DataFrame`类添加的方法，因此需要首先运行上面的单元。

In [63]:
import pandas as pd
df = pd.DataFrame(np.c_[x, y, y2], columns=["x", "sin", "sin+noise"])
df.iplot(x="x", y=["sin", "sin+noise"], dimensions=[600, 400], width=2, xTitle="x")

如果设置`iplot()`方法的`asFigure`参数为`True`，则返回可以使用`iplot()`函数绘图的图表对象：

In [64]:
fig = df.iplot(x="x", y=["sin", "sin+noise"], 
               dimensions=[600, 400], width=2, xTitle="x", asFigure=True)
type(fig)

上面的`fig`对象几乎和前面的`fig`字典相同，不过它还提供了通过属性访问键值的功能，例如下面是第一条曲线的绘图属性：

In [72]:
fig.data[0].line

{'color': 'rgba(255, 153, 51, 1.0)', 'dash': 'solid', 'width': 2}

下面修改第一条曲线的粗细之后调用`iplot()`函数输出图表：

In [73]:
fig.data[0].line.width = 4
iplot(fig)

参考链接：

* Plotly的Python API文档: https://plot.ly/python/
* cufflinks项目: https://github.com/santosjorge/cufflinks